In [1]:
import sqlite3
from keras.models import load_model
import keras.backend.tensorflow_backend as tb
tb._SYMBOLIC_SCOPE.value = True

from matplotlib.pyplot import imshow
import tqdm.notebook as tqdm

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.spatial import distance
import time
import uuid
import json
from imageio import imread
import random

from multiprocessing import Pool, Process

import pickle
import mysql.connector

import numpy as np

from FaceLoader import FaceLoader
from SocialProcessor import SocialProcessor

import vk_api
import threading

%matplotlib inline

Using TensorFlow backend.


In [2]:
class StoppableThread(threading.Thread):
    """
    Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition.
    
    """
    func = None
    args = {}

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    def run(self):
        self.args['thread'] = self
        self.func(**self.args)
    

In [3]:
conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")

In [4]:
model_path = 'keras-facenet-master/model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def aunt_vk():
    return input(), True


def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [6]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [7]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

000901


In [8]:
processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/')

In [9]:
def performVkProcessing(delay=0, count=100, thread=False):
    processor = SocialProcessor(conn, model)
    time.sleep(delay)
    while count>0:
        task = processor.getTask()
        user_id = task[0]
        query_id = task[-1]
        processor.delTask(query_id)
        
        processor.processVkUser(api, user_id)
        if thread.stopped():
            break
        count-=1

In [10]:
threads = []
threading.active_count()

5

In [15]:
t_num = 100
limits = 10
step = 3
params = [(i*step, limits) for i in range(t_num)]
for args in params:
    x = StoppableThread()
    x.func = performVkProcessing
    args = {"delay": args[0], "count": args[1]}
    x.args = args
    x.start()
    threads.append(x)


Added 0 rows
Added 2 rows


Exception in thread Thread-33:
Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-bdde2ce43cb7>", line 22, in run
    self.func(**self.args)
  File "<ipython-input-9-75dd3a4299f1>", line 5, in performVkProcessing
    task = processor.getTask()
  File "/Users/aleksandrdremov/PycharmProjects/GlobalScraper/SocialProcessor.py", line 150, in getTask
    c = self.connection.cursor()
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 539, in cursor
    self.handle_unread_result(prepared)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 712, in handle_unread_result
    raise errors.InternalError("Unread result found")
mysql.connector.errors.InternalError: Unread result found

Exception in thread Thread-38


Added 0 rows


Exception in thread Thread-39:
Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-bdde2ce43cb7>", line 22, in run
    self.func(**self.args)
  File "<ipython-input-9-75dd3a4299f1>", line 5, in performVkProcessing
    task = processor.getTask()
  File "/Users/aleksandrdremov/PycharmProjects/GlobalScraper/SocialProcessor.py", line 150, in getTask
    c = self.connection.cursor()
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 539, in cursor
    self.handle_unread_result(prepared)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 712, in handle_unread_result
    raise errors.InternalError("Unread result found")
mysql.connector.errors.InternalError: Unread result found



In [31]:
threading.active_count()

17


Added 131 rows


In [ ]:
for i in range(len(threads)):
    threads[i].stop()